# exploring text summaries

 - does not work
 - we will only use the metadata

In [1]:
# import libraries
import urllib
from bs4 import BeautifulSoup
import re

In [2]:
from urllib.request import urlopen
content = urlopen("https://www.hoberg-driesch.de/")

In [3]:
soup = BeautifulSoup(content, 'html.parser')

In [4]:
def maystrip(a):
    if a is None:
        return a
    else:
        return a.strip()

In [5]:
a_text = soup.find_all('p')
#2 Removing
y=[re.sub(r'<.+?>',r'',str(a)) for a in a_text]

In [6]:
y

['\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tWir bündeln unser Know How!\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tMit der Übertragung der Handelsaktivitäten unserer Einzelgesellschaften zur Hoberg &amp; Driesch Röhrenhandel GmbH präsentiert sich die Röhrengruppe um die Hoberg &amp; Driesch GmbH &amp; Co. KG nun noch kundennäher und leistungsstärker.\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t',
 'Wir setzen höchste Qualitätsmaßstäbe in der Beratung und den Arbeitsabläufen.']

In [7]:
s = "\n".join(y)

In [8]:
a_text

[<p class="header-subline">
 								
 									Wir bündeln unser Know How!
 								
 							</p>, <p class="header-bodytext">
 								
 									Mit der Übertragung der Handelsaktivitäten unserer Einzelgesellschaften zur Hoberg &amp; Driesch Röhrenhandel GmbH präsentiert sich die Röhrengruppe um die Hoberg &amp; Driesch GmbH &amp; Co. KG nun noch kundennäher und leistungsstärker.
 								
 							</p>, <p>Wir setzen höchste Qualitätsmaßstäbe in der Beratung und den Arbeitsabläufen.</p>]

In [9]:
t_all = soup.get_text()

In [11]:
# kill all script and style elements
for script in soup(["script", "style"]):
    script.decompose()    # rip it out

# get text
text = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)

print(text)

HOBERG & DRIESCH - Hoberg und Driesch GmbH & Co. KG
Suchbegriff
Suchen
Downloads
Sprache wählen
deen
UnternehmenProdukteAnwen­dungs­gebieteServicesKontakt
Aus 6 mach 1
Wir bündeln unser Know How!
Mit der Übertragung der Handelsaktivitäten unserer Einzelgesellschaften zur Hoberg & Driesch Röhrenhandel GmbH präsentiert sich die Röhrengruppe um die Hoberg & Driesch GmbH & Co. KG nun noch kundennäher und leistungsstärker.
Unser Leistungs­ver­sprechen
24 Stunden Service
Rund um die Uhr beweisen wir unsere Zuverlässigkeit.
mehr lesen
Made in Germany
Um Ihnen die größtmögliche Produktqualität zu garantieren...
mehr lesen
Starke Zusatzleistungen
So umfangreich wie unser Sortiment sind auch unsere Anarbeitungs­möglich­keiten.
mehr lesen
Breite Produktpalette
Unser Sortiment ist bekannt für seine Breite und Tiefe.
mehr lesen
Persönliche Betreuung
Jeder unserer Mitarbeiter kennt unsere Produkte ganz genau.
mehr lesen
Zentrales Lager
Damit wir Sie jederzeit und kurzfristig zufriedenstellen können.

these descriptions are not really useful 

In [12]:
import RAKE
rake = RAKE.Rake(RAKE.SmartStopList())
rake.run(text, minCharacters = 1, maxWords = 5, minFrequency = 1)

[('der beratung und den arbeitsabläufen', 24.5),
 ('wir freuen uns auf sie', 23.0),
 ('mehr lesen\nstarke zusatzleistungen', 15.2),
 ('mehr lesen\nqualität\nkontinuität', 15.2),
 ('mehr lesen\nkontakt gefällig', 15.2),
 ('hoberg und driesch gmbh &', 14.5),
 ('erreichen sie uns', 11.0),
 ('mehr lesen\nmade', 10.2),
 ('mehr lesen\nflexibel', 10.2),
 ('hoberg & driesch', 6.0),
 ('kg 2018\ntop', 4.0),
 ('termintreu', 1.0),
 ('persönlich', 1.0)]

In [13]:
import pyperclip
pyperclip.copy(text)


## extracting metadata

In [2]:
import pandas as pd
from urllib.request import urlopen
headers_dom = ['url', 'section', 'code']
dom = pd.read_csv('../data/raw/20181018 Domain+WZ2008 Hackathon INOBAS.csv', sep=';', names=headers_dom, encoding='latin-1', skiprows=1)

dom.head()

,url,section,code
0,www.hoberg-driesch.de,L,68.31
1,www.hgdf.de,K,64.20
2,www.gasag.de,D,35.30
3,www.ergodirekt.de,K,65.11
4,www.wahler.de,M,70.10


In [3]:
headers = {'User-Agent': 
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0 Safari/65.1.15'}    

In [7]:
from urllib.request import urlopen, Request
def extract_metadata(url):
    #print("normal:" + url)
    abstr = ""
    desc = ""
    keywords = ""
    crashed = False
    try:
        try:
            content = urlopen(Request("https://" + url, headers={'User-Agent': 'Mozilla'}), timeout = 10)
        except:
            try:
                content = urlopen(Request("http://" + url, headers={'User-Agent': 'Mozilla'}), timeout = 10)
            except:
                raise
        soup = BeautifulSoup(content, 'html.parser')
        abstr = ""
        desc = ""
        keywords = ""
        for meta in soup.findAll("meta"):
            if not meta.has_attr('content'):
                continue
            metaname = meta.get('name', '').lower()
            metaprop = meta.get('property', '').lower()
            if 'description' == metaname or metaprop.find("description")>0:
                desc = meta['content'].strip()
            if 'abstract' == metaname or metaprop.find("abstract")>0:
                abstr = meta['content'].strip()
            if 'keywords' == metaname or metaprop.find("keywords")>0:
                keywords = meta['content'].strip()
    except Exception as e: 
            print(e)
            print(url)
            crashed = True
    return (abstr, desc, keywords, crashed)

In [8]:
extract_metadata("www.hoberg-driesch.de")

('Qualitätsstahlrohre von Hoberg & Driesch. Von nahtlos bis grenzenlos.',
 'Hoberg & Driesch GmbH & Co. KG ist eines der führenden europäischen Großhandelshäuser für Stahlrohre mit Firmensitz in Düsseldorf und weiteren Niederlassungen in Hamburg, Kornwestheim, München, Dresden und Istanbul.',
 'Röhrengrosshandel, Starkwandrohre, Normalwandrohre, Gewinderohre, Kesselrohre, Hohlprofile, Präzisions\xadstahl\xadrohre, HPZ-Rohre, Wälzlager\xadstahl\xadrohre, Feinkornbaustahlrohre, Maschinenbaustahlrohre',
 False)

In [16]:
data = pd.DataFrame(dom["url"])
import numpy as np
datas = np.array_split(data, 20)
datas[0].shape

(1078, 1)

In [19]:
import dask.dataframe as dd
from dask.multiprocessing import get
data = pd.DataFrame(dom["url"])

for i in range(0,9):
    ddata = dd.from_pandas(datas[i], npartitions=60)

    res = ddata.map_partitions(lambda df: df.apply((lambda row: extract_metadata(*row)), axis=1)).compute(get=get)  
    
    res.to_csv("metadata" + str(i) + ".csv")

<urlopen error [Errno -3] Temporary failure in name resolution>
foo
<urlopen error [Errno -3] Temporary failure in name resolution>
foo
[Errno 104] Connection reset by peer
www.baywa.de
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.ukgm.de
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.aerzteverlag.de
<urlopen error [Errno -2] Name or service not known>
www.krankenhaus-gmbh.de
HTTP Error 308: Permanent Redirect
www.karl-mayer.de
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.bb-roesner-backstube.de
HTTP Error 404: Not Found
www.atosworldline.de
timed out
www.dvs-gruppe.de
hostname 'www.lande.de' doesn't match either of 'www.lsw.de', 'www.lsw-holding.de', 'www.lsw-netz.de'
www.lande.de
HTTP Error 403: Forbidden
www.wsw-energiedienstleistungen.de
<urlopen error timed out>
www.alcatel-lucent.de
timed out
www.airport.de
<urlopen error [Errno -2] Name or se

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


hostname 'jens.runne.barmenia.de' doesn't match either of '*.barmenia.de', 'barmenia.de'
www.jens.runne.barmenia.de
<urlopen error [Errno -2] Name or service not known>
www.dura-reiche.de
HTTP Error 403: Forbidden
www.reinert-logistic.de
The read operation timed out
www.putzmeister.de
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.mme.de
hostname 'www.birfood.de' doesn't match either of 'www.birgroup.de', 'birgroup.de'
www.birfood.de
<urlopen error timed out>
www.ejf-lazarus.de
HTTP Error 403: Forbidden
www.biotronik.de
HTTP Error 401: Unauthorized
www.massivholz-amco.de
HTTP Error 404: Not Found
www.hach-lange.de
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.emsland-group.de
HTTP Error 503: Service Unavailable
www.emsland-staerke.de
hostname 'www.borromäus-hospital-leer.de' doesn't match either of 'www.bonifatius-hospitalgesellschaft.de', 'akademie-franziskus.de', 'www.bonifatius-hospital-lingen

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


<urlopen error timed out>
www.hanning-hew.de
HTTP Error 404: Not Found
www.gwb-essen.de
HTTP Error 404: Not Found
www.sws-netz.de
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.kronprinz.de
<urlopen error [Errno -2] Name or service not known>
www.aral-fluessiggas.de
HTTP Error 500: Internal Server Error
www.wollschlaeger.de
HTTP Error 404: Not Found
www.advisa-herten.de
<urlopen error [Errno -3] Temporary failure in name resolution>
www.mhrheine.de
HTTP Error 400: Bad Request
www.rr-icecream.de
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.avo.de
timed out
www.rnr-net.de
timed out
www.kuchenmeister.de
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.tital.de
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.tetrapak.de
<urlopen error timed out>
www.totalwalther.de
<urlopen error timed out>
www.xchanging.de
<urlop

...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


HTTP Error 404: Unknown site!
www.handte.de
hostname 'www.feix-orthopaedie.de' doesn't match either of 'sanitaetshaus-feix.de', 'www.sanitaetshaus-feix.de'
www.feix-orthopaedie.de
HTTP Error 403: Forbidden
www.kbsch.de
<urlopen error [Errno -2] Name or service not known>
www.k-mueller-galabau.de
HTTP Error 404: 
www.langen-sondermann.de
<urlopen error [Errno -2] Name or service not known>
www.lsg-gebaeudereinigung.de
<urlopen error [Errno -2] Name or service not known>
www.fischer-bamberg.de


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.auto-soerensen.de
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.noris-blattgold.de
HTTP Error 403: Forbidden
www.iserlohner.de
HTTP Error 403: Forbidden
www.rats-apotheke-uslar.de
HTTP Error 500: Internal Server Error
www.vesteyfoods.de
Remote end closed connection without response
www.clausen-landtechnik.de
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.sanitaetshaus-strack.de
HTTP Error 404: Not Found
www.rspapierrollen.de
HTTP Error 404: Not Found
www.reva-energy.de
<urlopen error [Errno -2] Name or service not known>
www.schaaf-kalkulation.de
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.ruemker-gruppe.de
hostname 'www.symax-gmbh.de' doesn't match either of '*.teslagrohmannautomation.de', 'teslagrohmannautomation.de'
www.symax-gmbh.de
<urlopen error [Errno 

IndexError: list index out of range

In [23]:
len(datas)

10

In [22]:
list(range(0,19))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [9]:
import swifter

res = pd.DataFrame(dom["url"])
res[["abstract", "description", "keywords", "crashed"]] = res['url'].swifter.apply(extract_metadata).apply(pd.Series)

<urlopen error unknown url type: 0        http>
0        www.hoberg-driesch.de
1                  www.hgdf.de
2                 www.gasag.de
3            www.ergodirekt.de
4                www.wahler.de
5              www.msa-auer.de
6           www.nuernberger.de
7                 www.ds-cc.de
8            www.mlk-berlin.de
9             www.goldhofer.de
10        www.faber-castell.de
11    www.klinikum-landshut.de
12              www.biotest.de
13           www.vattenfall.de
14               www.klksig.de
15           www.dsv-gruppe.de
Name: url, dtype: object
[Errno 104] Connection reset by peer
www.baywa.de
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.ukgm.de
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:841)>
www.aerzteverlag.de
<urlopen error [Errno -2] Name or service not known>
www.krankenhaus-gmbh.de
HTTP Error 308: Permanent Redirect
www.karl-mayer.de


KeyboardInterrupt: 

In [ ]:
res["crashed"].value_counts()

In [91]:
content = urlopen("https://www.conrad.de")

HTTPError: HTTP Error 403: Forbidden